# 5장 텐서보드와 모델 재사용

학습시킨 모델을 저장하고 재사용하는 방법과 텐서플로의 가장 큰 장점인 텐서보드를 이용해 손실값의 변화를 그래프로 추적해보는 법을 배워보자

1. 데이터 파일을 만들었으면, 다음부터 데이터를 읽어 들이고 변환하는 코드로 프로그램을 시작한다.

In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data/data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [2]:
x_data

array([[0., 0.],
       [1., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [3]:
y_data

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [4]:
data

array([[0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.]], dtype=float32)

원시 데이터

In [5]:
data2 = np.loadtxt('./data/data.csv', delimiter=',',
                  unpack=False, dtype='float32')

In [6]:
data2

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0.],
       [1., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1.]], dtype=float32)

```py
unpack=True
```

In [7]:
data

array([[0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1.]], dtype=float32)

In [8]:
x_data

array([[0., 0.],
       [1., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

2. 이제 신경망 모델을 구성한다. 먼저 모델을 저장할 때 쓸 변수를 하나 만든다. 이 변수는 학습에 직접 사용되지는 않고, 학습 횟수를 카웅ㄴ트하는 변수이다. 이를 위해 변수 정의 시 trainable=False 옵션을 준다.

In [4]:
global_step = tf.Variable(0, trainable=False, name='global_step')

3. 이번에는 앞 장에서보다 계층을 하나 더 늘리고, 편향은 없이 가중치만 사용한 모델로 만들어보자.

In [5]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
       tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model)
       )

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

코드를 보면 두 번째 가중치인 W2의 형태가 [10, 20]인 것을 알 수 있는데, 그 이유는 앞단 계층의 출력 크기가 10이고, 뒷단 계층의 입력 크기가 20이기 때문이다. 신경망의 계층 늘리기는 이처럼 간단하게 처리할 수 있다.

** NOTE **
- 신경망의 계층 수와 은닉층의 뉴런 수(예제에서 [10, 20] 부분)을 늘리면 복잡도가 높은 문제를 해결하는 데 도움이 된다. 다만 이렇게 한다고 해서 무조건 도움이 되는 것은 아니며, 오히려 과적합이라는 문제에 빠질 수 있다.

- 즉, 신경망 모델 구성에서 계층과 뉴런 수를 최적화하는것이 효과적인 모델을 설계하는 핵심 중 하나이다.

마지막 줄에 보면 global_step 매개변수에 앞서 정의한 global_step 변수를 넘겨준 것을 확인할 수 있다. 이렇게 하면 최적화 함수가 학습용 변수들을 최적화할때마다 global_step 변수의 값을 1씩 증가시키게 된다.

In [6]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

tf.global_variables는 앞서 정의한 변수들을 가져오는 함수이다. 이 함수를 써서 앞서 정의한 변수들을 모두 가져와서, 이후 이 변수들을 파일에 저장하거나 이전에 학습한 결과를 불러와 담는 변수들로 사용한다.

5. 다음 코드는 ./model 디렉터리에 기존에 학습해둔 모델이 있는지를 확인해 보고, 모델이 있다면 saver.restore 함수를 사용해서 학습된 값들을 불러오고, 없다면 변수를 새로 초기화한다. 학습된 모델을 저장한 파일을 체크포인트 파일(checkpoint file)이라고 한다.

In [7]:
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-12


6. 간단하게 최적화를 수행한다. 이전과는 달리 step 값이 아니라 global_step 값을 이용해 학습을 몇 번째 진행하고 있는지를 출력해준다. global_step은 텐서 타입으 변수이므로 값을 가져올때 sess.run(global_step)을 이용해야 한다.

In [8]:
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print ('Step: %d, ' % sess.run(global_step),
           'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step: 13,  Cost: 0.664
Step: 14,  Cost: 0.655


In [9]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-14'

두 번째 매개변수는 체크포인트 파일의 위치와 이름이다. global_step의 값은 저장할 파일의 이름에 추가로 붙게 되며, 텐서 변수 또는 숫자값을 넣어줄 수 있다. 이를 이용해 여러 상태의 체크포인트를 만들 수 있고, 가장 효과적인 체크포인트를 선별해서 상요할 수 있다.

8. 이제 앞 장에서와 같이 예측 결과와 정확돌르 확인하는 다음 코드를 마지막으로 넣고 실행 결과를 확인해 보자

In [10]:
prediction = tf.arg_max(model, 1)
target = tf.argmax(Y, 1)
print ('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print ('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Instructions for updating:
Use `argmax` instead
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


2번을 시행하였지만, global_step으로 저장한 값을 불러와서 증가시켰으므로 Step이 3부터 시작했고, 정확도 또한 올라간 것을 볼 수 있다. 텐서플로를 이용하면 이렇게 쉽게 학습시킨 모델을 저장하고 불러와서 재사용할 수 있다.

또한 이 방식을 응용해 모델 구성, 학습, 예측 부분을 각각 분리하여 학습을 따로 한 뒤 예측만 단독으로 실행하는 프로그램을 작성할 수 있다.

## 전체 코드

In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data/data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

################
# 신경망 모델 구성 #
################

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step = global_step)

In [16]:
################
# 신경망 모델 학습 #
################

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model_2/')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print ("it's new, not resotred")
    
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print ('Step: %d, ' % sess.run(global_step),
           'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

saver.save(sess, './model_2/dnn.ckpt', global_step=global_step)

###########
# 결과 확인 #
###########

prediction = tf.arg_max(model, 1)
target = tf.argmax(Y, 1)
print ('예측값', sess.run(prediction, feed_dict={X:x_data}))
print ('실제값', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

INFO:tensorflow:Restoring parameters from ./model_2/dnn.ckpt-46
Step: 47,  Cost: 0.556
Step: 48,  Cost: 0.555
예측값 [0 1 2 0 0 2]
실제값 [0 1 2 0 0 2]
정확도: 100.00


In [12]:
saver

In [15]:
global_step

<tf.Variable 'global_step:0' shape=() dtype=int32_ref>

## 5.2 텐서보드 사용하기

- 모델을 효과적으로 실험하려면 학습 과정을 추적하는 일이 매우 중요하다.
- 텐서보드는 학습하는 중간중간 손실값이나 정확도 또는 겨과물로 나온 이미지나 사운드 파일들을 다양한 방식으로 시각화해 보여준다.

1. 먼저 데이터를 읽어 들이는 코드와 플레이스 홀더 값들을 똑같이 넣는다.

In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data/data.csv', delimiter=',', unpack=True, dtype='float32')

In [2]:
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [3]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

2. 각 신경망의 각 계층에 다음 코드를 덧붙이자.

In [4]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

- with tf.name\_scope로 묶은 블록은 텐서보드에서 한 계층 내부를 표혆나다. 그리고 변수 뒤에 name='W1'처럼 이름을 붙이면 텐서보드에서 해당 이름의 변수가 어디서 사용되는지 쉽게 확인할 수 있다. 이름은 변수뿐만 아니라 프렐이스 홀더, 각각의 연산, 활성화 함수 등 모든 텐서에 붙일 수 있다.

In [5]:
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost)

3. 손실값을 추적하기 위해 수집할 값을 지정하는 코드를 작성한다

```py
tf.summary.scalar('cost', cost)
```

tf.summary 모듈의 scalar 함수는 값이 하나인 텐서를 수집할때 사용한다. scalar 이외에도 histogram, image, audio 등이 있다.

4. 모델을 불러들이거나 초기화하는 코드를 넣는다.

In [6]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model_3')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

5. tf.summary.merge_all 함수로 앞서 지정한 텐서들을 수집한 다음 tf.summary.FileWriter 함수를 사용해 그래프와 텐서들의 값을 저장할 디렉토리를 설정한다.

In [7]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

6. 최적화를 실행하는 코드를 작성하자

In [8]:
for step in range(100):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    
    print ('Step: %d, ' % sess.run(global_step),
           'Cost: %.3f' % sess.run(cost, feed_dict={X:x_data, Y:y_data}))
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

Step: 1,  Cost: 0.799
Step: 2,  Cost: 0.674
Step: 3,  Cost: 0.593
Step: 4,  Cost: 0.571
Step: 5,  Cost: 0.562
Step: 6,  Cost: 0.556
Step: 7,  Cost: 0.552
Step: 8,  Cost: 0.550
Step: 9,  Cost: 0.550
Step: 10,  Cost: 0.549
Step: 11,  Cost: 0.549
Step: 12,  Cost: 0.549
Step: 13,  Cost: 0.549
Step: 14,  Cost: 0.549
Step: 15,  Cost: 0.549
Step: 16,  Cost: 0.549
Step: 17,  Cost: 0.549
Step: 18,  Cost: 0.549
Step: 19,  Cost: 0.549
Step: 20,  Cost: 0.549
Step: 21,  Cost: 0.549
Step: 22,  Cost: 0.549
Step: 23,  Cost: 0.549
Step: 24,  Cost: 0.549
Step: 25,  Cost: 0.549
Step: 26,  Cost: 0.549
Step: 27,  Cost: 0.549
Step: 28,  Cost: 0.549
Step: 29,  Cost: 0.549
Step: 30,  Cost: 0.549
Step: 31,  Cost: 0.549
Step: 32,  Cost: 0.549
Step: 33,  Cost: 0.549
Step: 34,  Cost: 0.549
Step: 35,  Cost: 0.549
Step: 36,  Cost: 0.549
Step: 37,  Cost: 0.549
Step: 38,  Cost: 0.549
Step: 39,  Cost: 0.549
Step: 40,  Cost: 0.549
Step: 41,  Cost: 0.549
Step: 42,  Cost: 0.549
Step: 43,  Cost: 0.549
Step: 44,  Cost: 0.5

7. sess.run 을 이용해 앞서 merged로 모아둔 텐서의 값들을 계산해서 수집한 뒤, writer.add_summary 함수를 이용해 해당 값들을 앞서 지정한 디렉토리에 저장한다. 적절한 시점에 값들을 수집하고 저장하면 되며(여기서는 매 단계), 나중에 확인할 수 있도록 global_step 값을 이용해 수집한 시점을 기록해 둔다.

```py
summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
writer.add_summary(summary, global_step=sess.run(global_step))
```

8. 마지막으로 모델을 저장하고 예측하는 부분을 똑같이 작성한다.

In [9]:
saver.save(sess, './model_3/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print ('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print ('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


### 전체 코드

In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data/data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

##################
# 신경망의 모델 구성 #
#################

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost)

In [13]:
#################
# 신경망 모델 학습 #
###############

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model_4')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print ('Step: %d, ' % sess.run(global_step),
           'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    
saver.save(sess, './model_4/dnn.ckpt', global_step=global_step)

INFO:tensorflow:Restoring parameters from ./model_4/dnn.ckpt-900
Step: 901,  Cost: 0.549
Step: 902,  Cost: 0.549
Step: 903,  Cost: 0.549
Step: 904,  Cost: 0.549
Step: 905,  Cost: 0.549
Step: 906,  Cost: 0.549
Step: 907,  Cost: 0.549
Step: 908,  Cost: 0.549
Step: 909,  Cost: 0.549
Step: 910,  Cost: 0.549
Step: 911,  Cost: 0.549
Step: 912,  Cost: 0.549
Step: 913,  Cost: 0.549
Step: 914,  Cost: 0.549
Step: 915,  Cost: 0.549
Step: 916,  Cost: 0.549
Step: 917,  Cost: 0.549
Step: 918,  Cost: 0.549
Step: 919,  Cost: 0.549
Step: 920,  Cost: 0.549
Step: 921,  Cost: 0.549
Step: 922,  Cost: 0.549
Step: 923,  Cost: 0.549
Step: 924,  Cost: 0.549
Step: 925,  Cost: 0.549
Step: 926,  Cost: 0.549
Step: 927,  Cost: 0.549
Step: 928,  Cost: 0.549
Step: 929,  Cost: 0.549
Step: 930,  Cost: 0.549
Step: 931,  Cost: 0.549
Step: 932,  Cost: 0.549
Step: 933,  Cost: 0.549
Step: 934,  Cost: 0.549
Step: 935,  Cost: 0.549
Step: 936,  Cost: 0.549
Step: 937,  Cost: 0.549
Step: 938,  Cost: 0.549
Step: 939,  Cost: 0.549

'./model_4/dnn.ckpt-1000'

In [14]:
###########
# 결과 확인 #
##########

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)

print ('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print ('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
